In [ ]:
# Import Libraries
import requests
from pprint import PrettyPrinter
pp = PrettyPrinter()
import pandas as pd
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("TMDB_API_KEY")
omdb_api_key = os.getenv("OMDB_API_KEY")

Step 1 :Get List of all bollywood movies release afer 2010


In [ ]:
def get_movie_list()->pd.DataFrame:
    url = f"https://api.themoviedb.org/3/discover/movie?api_key={api_key}&include_adult=false&include_video=false&sort_by=revenue.desc&with_original_language=hi"
    response = requests.get(url).json()
    total_pages = response['total_pages']
    movies_info = pd.DataFrame()
    for page in range(1,total_pages):
        page_url  = f"{url}&page={page}"
        page_response = requests.get(page_url).json()['results']
        movies_info =pd.concat([movies_info,pd.DataFrame(page_response)])

    movies_info = movies_info[(movies_info['release_date']>='2000-01-01') & (movies_info['release_date']<='2024-04-01')]
    columns_to_keep = ['title','genre_ids','id','popularity','release_date']
    movies_info = movies_info[columns_to_keep]
    return movies_info

In [ ]:
movies_info = get_movie_list()
movies_info=movies_info.reset_index()
movies_info.shape

In [ ]:
def get_movie_details(movie_id): 
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}"
    response = requests.get(url).json()
    return response

In [ ]:
useful_columns = ['budget','id','imdb_id','popularity','revenue','title']
movie_details = pd.DataFrame()
for i in range(len(movies_info)):
    response = get_movie_details(movies_info['id'][i])
    for key,value in list(response.items()):
        if key not in useful_columns:
            del response[key]
    movie_details= pd.concat([movie_details,(pd.DataFrame([response]))])

In [ ]:
movie_details.shape 

In [ ]:
movie_details_filtered = movie_details[~((movie_details['revenue']==0) | (movie_details['budget']==0))]
movie_details_filtered.shape

In [ ]:
first_level_info = movie_details_filtered.merge(movies_info[['id','release_date']],on='id',how='left')
first_level_info.head()

In [ ]:
def get_movie_credits(movie_id):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={api_key}"
    response = requests.get(url).json()
    return response

In [ ]:
def get_cast_crew_popularity(interest_area:str,movie_id:int)->float:
    cast_name =[]
    cast_details = get_movie_credits(movie_id)
    cast_popularity = 0
    for i in range(len(cast_details[interest_area])):
        if cast_details[interest_area][i]['name'] in cast_name:
            cast_popularity += 0
        else:
            cast_popularity += cast_details[interest_area][i]['popularity']
            cast_name.append(cast_details[interest_area][i]['name'])
    return cast_popularity

print(get_cast_crew_popularity('cast',61202))
get_cast_crew_popularity('crew',61202)

In [ ]:
actors_star_power = {
    'Aamir Khan':[15480],
    'Salman Khan':[16045],
    'Shah Rukh Khan':[17705],
    'Akshay Kumar':[14730],
    'Ajay Devgn':[13760],
    'Hrithik Roshan':[12680],
    'Ranbir Kapoor':[5260]
}

def get_star_power(movie_id):
    cast_details = get_movie_credits(movie_id)
    cast_names = []
    for i in range(len(cast_details['cast'])):
        cast_names.append(cast_details['cast'][i]['name'])
    count = 0
    for name in cast_names:
        if name in actors_star_power.keys():
            count+=1
        else:
            count +=0
    if count >=1:
        star_present = 'Yes'
    else:
        star_present = 'No'
    return star_present

In [ ]:
cast_pop_dict ={'id':[],'cast_popularity':[],'crew_popularity':[],'star_present':[]}
for i in range(len(first_level_info)):
    cast_popularity = get_cast_crew_popularity('cast',first_level_info['id'][i])
    crew_popularity = get_cast_crew_popularity('crew',first_level_info['id'][i])
    star_present = get_star_power(first_level_info['id'][i])
    cast_pop_dict['id'].append(first_level_info['id'][i])
    cast_pop_dict['cast_popularity'].append(cast_popularity)
    cast_pop_dict['crew_popularity'].append(crew_popularity)
    cast_pop_dict['star_present'].append(star_present)
cast_popularity_df = pd.DataFrame(cast_pop_dict)

In [ ]:
second_level_info = first_level_info.merge(cast_popularity_df,on='id',how='left')
second_level_info.head()

In [ ]:
second_level_info.to_parquet('basic_features.parquet')